In [1]:
from copy import deepcopy
"""
See the bug below
"""
class ExistingAttributeError(Exception):
    pass

class Inject:
    cache = []

    def __init__(self, *args, **kwargs):
        raise ValueError("Do not create an instance of this class")

    def __init_subclass__(
                cls,
                *,
                to: list or any,
                for_package: any or None = None,
                overwrite:bool = False
            ):

        if not isinstance(to, list):
            to = [to]

        attrs = [
                    (k,v) for k,v in cls.__dict__.items()
                if k not in ['__module__', '__doc__']
            ]

        for name, attr in attrs:
            for target in to:
                if overwrite == False and hasattr(target, name) and name not in Inject.cache:
                    raise ExistingAttributeError(
                            f"Class '{target.__name__}' already has attribute, '{name}'. "
                            "Pass `overwrite = True` to avoid this error."
                        )
                setattr(target, name, attr)
                Inject.cache.append(name)
                if for_package:
                    def temp(obj, *args, **kwargs):
                        return getattr(obj, name)(*args, **kwargs)
                    setattr(for_package, name, deepcopy(temp))
                delattr(cls, name)

In [2]:
import pandas as pd

In [3]:
class _(Inject, to=pd.DataFrame, for_package=pd):
    
    def show_cols(self):
        return list(self.columns)

    def column_list_combined(self, other_df):
        return list(list(self.columns) + list(other_df.columns))

In [4]:
df1 = pd.DataFrame({'a': [], 'b': []})
df2 = pd.DataFrame({'c': [], 'd': []})

# df1.column_list_combined(df2)

In [5]:
pd.show_cols(df1)

TypeError: _.column_list_combined() missing 1 required positional argument: 'other_df'

In [ ]:
pd.say_hi(df, "hey")

hihey
